# **Match ente imágenes filtradas y sus multilabels**

## **Cargar targets**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
labels_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/train_v2_dehazed.csv', sep=';')
labels_df.head()

,image_name,tags
0,train_dehazed_0,haze primary
1,train_dehazed_1,agriculture clear primary water
2,train_dehazed_2,clear primary
3,train_dehazed_3,clear primary
4,train_dehazed_4,agriculture clear habitation primary road


In [ ]:
labels_df.groupby('tags').count().reset_index().sort_values('image_name',ascending=False).head(5)

,tags,image_name
369,clear primary,13636
433,partly_cloudy primary,3630
386,cloudy,2089
382,clear primary water,1850
143,agriculture clear primary road,1680


In [ ]:
labels_df.shape

(40479, 2)

## **create_tag_mapping**

In [ ]:
labels = set()
for i in range(len(labels_df)):
	tags = labels_df['tags'][i].split(' ')
	labels.update(tags)
labels

{'agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water'}

In [ ]:
labels = list(labels)
labels.sort()
labels

['agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water']

In [ ]:
tag_mapping = {labels[i]:i for i in range(len(labels))}
tag_mapping

{'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}

In [ ]:
inv_tag_mapping = {i:labels[i] for i in range(len(labels))}
inv_tag_mapping

{0: 'agriculture',
 1: 'artisinal_mine',
 2: 'bare_ground',
 3: 'blooming',
 4: 'blow_down',
 5: 'clear',
 6: 'cloudy',
 7: 'conventional_mine',
 8: 'cultivation',
 9: 'habitation',
 10: 'haze',
 11: 'partly_cloudy',
 12: 'primary',
 13: 'road',
 14: 'selective_logging',
 15: 'slash_burn',
 16: 'water'}

## **create_file_mapping**

In [ ]:
mapping = dict()

for i in range(len(labels_df)):
		name, tags = labels_df['image_name'][i], labels_df['tags'][i]
		mapping[name] = tags.split(' ')
  
mapping

{'train_dehazed_0': ['haze', 'primary'],
 'train_dehazed_1': ['agriculture', 'clear', 'primary', 'water'],
 'train_dehazed_2': ['clear', 'primary'],
 'train_dehazed_3': ['clear', 'primary'],
 'train_dehazed_4': ['agriculture', 'clear', 'habitation', 'primary', 'road'],
 'train_dehazed_5': ['haze', 'primary', 'water'],
 'train_dehazed_6': ['agriculture',
  'clear',
  'cultivation',
  'primary',
  'water'],
 'train_dehazed_7': ['haze', 'primary'],
 'train_dehazed_8': ['agriculture', 'clear', 'cultivation', 'primary'],
 'train_dehazed_9': ['agriculture', 'clear', 'cultivation', 'primary', 'road'],
 'train_dehazed_10': ['agriculture',
  'clear',
  'primary',
  'slash_burn',
  'water'],
 'train_dehazed_11': ['clear', 'primary', 'water'],
 'train_dehazed_12': ['cloudy'],
 'train_dehazed_13': ['clear', 'primary'],
 'train_dehazed_14': ['cloudy'],
 'train_dehazed_15': ['clear', 'primary'],
 'train_dehazed_16': ['clear', 'primary'],
 'train_dehazed_17': ['partly_cloudy', 'primary'],
 'train_deh

## **one_hot_encode**

In [ ]:
# from numpy import zeros
# encoding = zeros(len(mapping), dtype='uint8')
# encoding

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [ ]:
# for tag in labels:
#   encoding[mapping[tag]] = 1
# #encoding

## **Cargar imagenes**

In [ ]:
photos  = list()
targets = list()

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [ ]:
# ruta = '/content/drive/MyDrive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/train-dehazed-jpg/'
range(labels_df.shape[0])

range(0, 40479)

In [ ]:

from tqdm import tqdm
from numpy import zeros

for i in tqdm(range(40479)):    
    filename = 'train_dehazed_{}.jpg'.format(i)

  # load image
    photo = load_img('/content/drive/MyDrive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/train-dehazed-jpg/'+filename, target_size=(64,64))
    photo = img_to_array(photo, dtype='uint8')

  # get tags
    tags = mapping[filename[:-4]]

  # one hot encode tags
    encoding = zeros(len(tag_mapping), dtype='uint8')
    for tag in tags:
      encoding[tag_mapping[tag]] = 1
    # target = encoding

  # store
    photos.append(photo)
    targets.append(encoding)
	

 96%|█████████▋| 39038/40479 [7:27:29<15:57,  1.50it/s]

UnidentifiedImageError: ignored

In [ ]:
X = asarray(photos, dtype='uint8')
y = asarray(targets, dtype='uint8')
print(X.shape, y.shape)

In [ ]:
savez_compressed('/content/drive/MyDrive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/planet_data.npz', X, y)

In [ ]:
from numpy import asarray

X = asarray(photos, dtype='uint8')
y = asarray(targets, dtype='uint8')
print(X.shape, y.shape)

(39038, 64, 64, 3) (39038, 17)


In [ ]:
from numpy import savez_compressed

savez_compressed('/content/drive/MyDrive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/planet_data_64.npz', X, y)

In [ ]:
photos2  = list()
targets2 = list()

In [ ]:

from tqdm import tqdm
from numpy import zeros

for i in tqdm(range(39039, 40479)):    
    filename = 'train_dehazed_{}.jpg'.format(i)

  # load image
    photo = load_img('/content/drive/MyDrive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/train-dehazed-jpg/'+filename, target_size=(64,64))
    photo = img_to_array(photo, dtype='uint8')

  # get tags
    tags = mapping[filename[:-4]]

  # one hot encode tags
    encoding = zeros(len(tag_mapping), dtype='uint8')
    for tag in tags:
      encoding[tag_mapping[tag]] = 1
    # target = encoding

  # store
    photos2.append(photo)
    targets2.append(encoding)
	


100%|██████████| 1440/1440 [18:03<00:00,  1.33it/s]


In [ ]:
from numpy import asarray

X_2 = asarray(photos2, dtype='uint8')
y_2 = asarray(targets2, dtype='uint8')
print(X_2.shape, y_2.shape)

(1440, 64, 64, 3) (1440, 17)


In [ ]:
from numpy import savez_compressed
savez_compressed('/content/drive/MyDrive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/planet_data_64_2.npz', X_2, y_2)

In [ ]:
photos12  = list()
targets12 = list()

In [ ]:

from tqdm import tqdm
from numpy import zeros

for i in tqdm(range(39038, 39039)):    
    filename = 'train_dehazed_{}.jpg'.format(i)

  # load image
    photo = load_img('/content/drive/MyDrive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/train-dehazed-jpg/'+filename, target_size=(64,64))
    photo = img_to_array(photo, dtype='uint8')

  # get tags
    tags = mapping[filename[:-4]]

  # one hot encode tags
    encoding = zeros(len(tag_mapping), dtype='uint8')
    for tag in tags:
      encoding[tag_mapping[tag]] = 1

  # store
    photos12.append(photo)
    targets12.append(encoding)

  0%|          | 0/1 [00:00<?, ?it/s]


UnidentifiedImageError: ignored

In [ ]:
from numpy import asarray

X_12 = asarray(photos12, dtype='uint8')
y_12 = asarray(targets12, dtype='uint8')
print(X_12.shape, y_12.shape)

(0,) (0,)


In [ ]:

from tqdm import tqdm
from numpy import zeros

for i in tqdm(range(40479)):    
    filename = 'train_dehazed_{}.jpg'.format(i)

  # load image
    photo = load_img('/content/drive/MyDrive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/train-dehazed-jpg/'+filename, target_size=(128,128))
    photo = img_to_array(photo, dtype='uint8')

  # get tags
    tags = mapping[filename[:-4]]

  # one hot encode tags
    encoding = zeros(len(tag_mapping), dtype='uint8')
    for tag in tags:
      encoding[tag_mapping[tag]] = 1
    # target = encoding

  # store
    photos.append(photo)
    targets.append(encoding)
	

In [ ]:
X_128 = asarray(photos, dtype='uint8')
y_128 = asarray(targets, dtype='uint8')
 print(X_128.shape, y_128.shape)

In [ ]:
# save 
savez_compressed('/content/drive/MyDrive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/planet_data_128.npz', X_128, y_128)

In [ ]:

from tqdm import tqdm
from numpy import zeros

for i in tqdm(range(40479)):    
    filename = 'train_dehazed_{}.jpg'.format(i)

  # load image
    photo = load_img('/content/drive/MyDrive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/train-dehazed-jpg/'+filename, target_size=(256,256))
    photo = img_to_array(photo, dtype='uint8')

  # get tags
    tags = mapping[filename[:-4]]

  # one hot encode tags
    encoding = zeros(len(tag_mapping), dtype='uint8')
    for tag in tags:
      encoding[tag_mapping[tag]] = 1
    # target = encoding

  # store
    photos.append(photo)
    targets.append(encoding)
	

In [ ]:
X_256 = asarray(photos, dtype='uint8')
y_256 = asarray(targets, dtype='uint8')
print(X_256.shape, y_256.shape)

In [ ]:
# save 
savez_compressed('/content/drive/MyDrive/Colab Notebooks/DiplomadoIA/M1_AplicacionesConVisionArtificial/Proyecto_Amazonia/Datos_Entrenamiento/planet_data_256.npz', X_256, y_256)